# Part 5 - Intro to Encrypted Programs

Believe it or not, it is possible to compute with encrypted data. In other words, it's possible to run a program where ALL of the variables in the program are encrypted! 

In this tutorial, we're going to walk through very basic tools of encrypted computation. In particular, we're going to focus on one popular approach called Secure Multi-Party Computation. In this lesson, we'll learn how to build an encrypted calculator which can perform calculations on encrypted numbers.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/)

References: 
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

# Step 1: Encryption Using Secure Multi-Party Computation

SMPC is at first glance a rather strange form of "encryption". Instead of using a public/private key to encrypt a variable, each value is split into multiple "shares", each of which operate like a private key. Typically, these "shares" will be distributed amongst 2 or more "owners". Thus, in order to decrypt the variable, all owners must agree to allow the decryption. In essence, everyone has a private key.

### Encrypt()

So, let's say we wanted to "encrypt" a varible "x", we could do so in the following way.

In [1]:
Q = 1234567891011

In [2]:
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(0,Q)
    share_b = random.randint(0,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(508598083450, 1210525625153, 750012073444)

As you can see here, we have split our variable "x" into 3 different shares, which could be sent to 3 different owners.

### Decrypt()

If we wanted to decrypt these 3 shares, we could simply sum them together and take the modulus of the result (mod Q)

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)

In [7]:
decrypt(a, b, c)

25

Importantly, notice that if we try to decrypt with only two shares, the decryption does not work!

In [8]:
decrypt(a, b)

1073754042472

Thus, we need all of the owners to participate in order to decrypt the value. It is in this way that the "shares" act like private keys, all of which must be present in order to decrypt a value.

# Step 2: Basic Arithmetic Using SMPC

However, the truly extraordinary property of Secure Multi-Party Computation is the ability to perform computation **while the variables are still encrypted**. Let's demonstrate simple addition below. 

In [9]:
x = encrypt(25)
y = encrypt(5)

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

### Success!!!

And there you have it! If each worker (separately) adds their shares together, then the resulting shares will decrypt to the correct value (25 + 5 == 30). 

As it turns out, SMPC protocols exist which can allow this encrypted computation for the following operations:
- addition (which we've just seen)
- multiplication
- comparison

and using these basic underlying primitives, we can perform arbitrary computation!!!

In the next section, we're going to learn how to use the PySyft library to perform these operations!

# Step 3: SMPC Using PySyft

In the previous sections, we outlined some basic intuitions around SMPC is supposed to work. However, in practice we don't want to have to hand-write all of the primitive operations ourselves when writing our encrypted programs. So, in this section we're going to walk through the basics of how to do encrypted computation using PySyft. In particualr, we're going to focus on how to do the 3 primitives previously mentioned: addition, multiplication, and comparison.

First, we need to create a few Virtual Workers (which hopefully you're now familiar with given our previous tutorials). 

In [12]:
import syft as sy
hook = sy.TorchHook()

bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")
bill = sy.VirtualWorker(id="bill")

### Basic Encryption/Decryption

Encryption is as simple as taking any PySyft tensor and calling .share(). Decryption is as simple as calling .get() on the shared variable

In [13]:
x = sy.LongTensor([25])

In [14]:
encrypted_x = x.share(bob, alice, bill)

In [15]:
encrypted_x.get()


 25
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

### Introspecting the Encrypted Values

If we look closer at Bob, Alice, and Bill's workers, we can see the shares that get created!

In [16]:
bob._objects

{}

In [17]:
x = sy.LongTensor([25]).share(bob, alice,bill)

In [18]:
bob._objects

{30897196848: [_LocalTensor - id:30897196848 owner:bob]}

In [19]:
# Bob's share
bobs_share = list(bob._objects.values())[0].parent[0]
bobs_share

2058802259

In [20]:
# Alice's share
alices_share = list(alice._objects.values())[0].parent[0]
alices_share

-2034801769

In [21]:
# Bill's share
bills_share = list(bill._objects.values())[0].parent[0]
bills_share

-24000465

And if we wanted to, we could decrypt these values using the SAME approach we talked about earlier!!!

In [22]:
Q = sy.spdz.spdz.field

(bobs_share + alices_share + bills_share) % Q

25

As you can see, when we called .share() it simply split the value into 3 shares and sent one share to each of the parties!

# Encrypted Arithmetic

And now you see that we can perform arithmetic on the underlying values! The API is constructed so that we can simply perform arithemtic like we would normal PyTorch tensors.

Note: for comparison, it returns boolean outputs (true/false) in the form of Integers (1/0). 1 corresponds to True. 0 corresponds to False.

In [23]:
x = sy.LongTensor([25]).share(bob,alice)
y = sy.LongTensor([5]).share(bob,alice)

In [24]:
z = x + y
z.get()


 30
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [25]:
z = x * y
z.get()


 125
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [26]:
z = x > y
z.get()


 1
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [27]:
z = x < y
z.get()


 0
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [28]:
z = x == y
z.get()


 0
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [29]:
z = x == y + 20
z.get()


 1
[syft.core.frameworks.torch.tensor.LongTensor of size 1]